### Imports

In [ ]:
import sys

sys.path.append("../..")

from pathlib import Path
import pydicom
import os
import numpy as np
import matplotlib.pyplot as plt
import dicom_image_tools as dit

from dicom_image_tools.roi.square_roi import SquareRoi
from dicom_image_tools.helpers.voxel_data import (
    VoxelData,
)
from dicom_image_tools.helpers.point import (
    Point,
)

from utils.plot_utils import (
    _remove_ax_ticks,
    _create_colorbar,
)

from utils.dit_utils import (
    _fetch_roi_trace,
    _find_images_lims,
    _get_image_center,
    _calculate_image_lims_exclude_cornerbox,
)

In [ ]:
clinics_path_processed = (
    Path("C:/")
    / "Users"
    / "maxh01"
    / "Region Västerbotten"
    / "Strålningsfysik - Dokument"
    / "Radiologi"
    / "Modaliteter"
    / "FTV"
    / "Nya sensorer 2022"
    / "Nya sensorer 2022 parsed"
)

clininic_name = "ODR - Ersättningssensorer - 240814_parsed"
lab = "ODR Lab 3"
tube = 248056
# sensor = 25017921

# clininic_name = "ODR - gamla sensorer - 230327_parsed"
# lab = "ODR Lab 3"
# tube = 248056
# sensor = 24008597

mode = "export"

sensor_IDs = []
for item in (clinics_path_processed / clininic_name / lab / str(tube) / "Kalibrering").iterdir():
    sensor_IDs.append(item.name)

for ID in sensor_IDs:
    print(ID)

# sensor_IDs

In [ ]:
# for sensor in ['24008597']:
for sensor in sensor_IDs:
    data_path = clinics_path_processed / clininic_name / lab / str(tube) / "Kalibrering" / str(sensor)

    data = dit.import_dicom_from_folder(data_path)

    for study in data:
        for serie in data[study].Series:
            serie.import_image()
            serie.sort_images_on_acquisition_time()

    cmap_magnitude_image = plt.cm.binary
    lims = _find_images_lims(ImageVolumes=serie.ImageVolume)
    for study in data:
        for serie in data[study].Series:
            nr_images = len(serie.ImageVolume)
            fig, axs = plt.subplots(
                ncols=nr_images,
                nrows=1,
                figsize=(20, 3),
            )
            # _remove_ax_ticks(axs)

            for i in range(nr_images):
                ax = axs[i]
                image = serie.ImageVolume[i]

                cmap = cmap_magnitude_image
                lims = _find_images_lims(ImageVolumes=serie.ImageVolume)
                im = ax.imshow(image, cmap=cmap, vmin=lims[0], vmax=lims[1])
                ax.set(xlabel=f"{int(serie.ms[i])}ms {int(serie.kV[i])}kV")
                ax.set(title=f"T: {tube}, \n S: {serie.CompleteMetadata[i].DetectorID}")
                roi = SquareRoi(
                    height=100,
                    width=100,
                    center=_get_image_center(image=image),
                    pixel_size=serie.VoxelData[i],
                    roi_size_in_pixels=True,
                )

                trace_x, trace_y = _fetch_roi_trace(roi)
                ax.plot(trace_x, trace_y, "r")

            _create_colorbar(
                fig,
                im,
                ax=axs[0],
                binary=False,
                width="100%",
                height="20%",
                loc="lower left",
                bbox_to_anchor=(-0.2, -0.5, 1.185 * nr_images, 1),
                borderpad=0,
                orientation="horizontal",
            )

            plt.show()

    fig, axs = plt.subplots(ncols=3, figsize=(15, 5))

    for study in data:
        for serie in data[study].Series:
            roi_mean = [roi.get_mean(image) for image in serie.ImageVolume]
            dose_mGy = [serie.CompleteMetadata[i].EntranceDoseInmGy for i in range(len(serie.CompleteMetadata))]

            sensor_ID = serie.CompleteMetadata[0].DetectorID

            ax = axs[0]
            im = ax.plot(serie.ms, roi_mean, "--*", label=f"{serie.kV[0]} kV")
            ax.set(
                xlabel="set exposure time / ms",
                ylabel="ROI mean pixel value",
                title=f"tube: {tube}, \n sensor: {sensor_ID}",
            )
            ax.legend()

            ax = axs[1]
            im = ax.plot(dose_mGy, roi_mean, "--*", label=f"{serie.kV[0]} kV")
            ax.set(
                xlabel="Measured dose [mGy]",
                ylabel="ROI mean pixel value",
                title=f"tube: {tube}, \n sensor: {sensor_ID}",
            )
            ax.legend()

            ax = axs[2]
            im = ax.plot(serie.ms, dose_mGy, "--*", label=f"{data[study].Series[0].kV[0]} kV")
            ax.set(
                xlabel="set exposure time / ms",
                ylabel="Measured dose [mGy]",
                title=f"tube: {tube}, \n sensor: {sensor_ID}",
            )
            ax.legend()

    for study in data:
        for serie in data[study].Series:
            for i in range(len(serie.ImageVolume)):
                image = serie.ImageVolume[i]
                lims = _calculate_image_lims_exclude_cornerbox(image)

                label_ms = f"{int(serie.ms[i])}ms "
                label_kv = f"{int(serie.kV[i])}kV "
                label_tube = f"tube: {tube} "
                label_id = f"sensor: {sensor_ID}"

                fig, ax = plt.subplots(figsize=(20, 15))
                im = ax.imshow(image, cmap=cmap, vmin=lims[0], vmax=lims[1], interpolation=None)

                ax.set(xlabel=label_ms + label_kv + label_tube + label_id)

                _create_colorbar(fig, im, ax, width="1%", bbox_to_anchor=(0.05, 0, 1, 1))

                print(serie.CompleteMetadata[0].DetectorID, serie.CompleteMetadata[0].DateOfLastDetectorCalibration)

                if mode == "plot":
                    plt.show()

                if mode == "export":
                    res_path = data_path / "png"
                    file_name = f"{sensor_ID}_{label_kv}_{label_ms}.png"
                    res_path.mkdir(exist_ok=True)
                    plt.savefig(res_path / file_name)
                    plt.close()